<a href="https://colab.research.google.com/github/BenjaAR/LA04_EXAMEN/blob/main/LAB04_AQUINO_REYES.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**LABORATORIO CALIFICADO**

SEMANA 4: INTEGRACIÓN Y LIMPIEZA. TABLA DE VARIABLES Y CREACIÓN DE NUEVAS VARIABLES. TRANSFORMACIÓN DE DATOS

Alumno: Aquino Reyes Benjy Alvaro


In [11]:
import pandas as pd
from ucimlrepo import fetch_ucirepo
from ucimlrepo import fetch_ucirepo

In [30]:
from ucimlrepo import fetch_ucirepo

# fetch dataset
breast_cancer_wisconsin_original = fetch_ucirepo(id=15)

# data (as pandas dataframes)
X = breast_cancer_wisconsin_original.data.features
y = breast_cancer_wisconsin_original.data.targets

# variable information
breast_cancer_wisconsin_original.variables


,name,role,type,demographic,description,units,missing_values
0,Sample_code_number,ID,Categorical,None,None,None,no
1,Clump_thickness,Feature,Integer,None,None,None,no
2,Uniformity_of_cell_size,Feature,Integer,None,None,None,no
3,Uniformity_of_cell_shape,Feature,Integer,None,None,None,no
4,Marginal_adhesion,Feature,Integer,None,None,None,no
5,Single_epithelial_cell_size,Feature,Integer,None,None,None,no
6,Bare_nuclei,Feature,Integer,None,None,None,yes
7,Bland_chromatin,Feature,Integer,None,None,None,no
8,Normal_nucleoli,Feature,Integer,None,None,None,no
9,Mitoses,Feature,Integer,None,None,None,no


In [33]:
# 3. Unir en un solo DataFrame
df_01 = pd.concat([X, y], axis=1)

df_01

,Clump_thickness,Uniformity_of_cell_size,Uniformity_of_cell_shape,Marginal_adhesion,Single_epithelial_cell_size,Bare_nuclei,Bland_chromatin,Normal_nucleoli,Mitoses,Class
0,5,1,1,1,2,1.0,3,1,1,2
1,5,4,4,5,7,10.0,3,2,1,2
2,3,1,1,1,2,2.0,3,1,1,2
3,6,8,8,1,3,4.0,3,7,1,2
4,4,1,1,3,2,1.0,3,1,1,2
...,...,...,...,...,...,...,...,...,...,...
694,3,1,1,1,3,2.0,1,1,1,2
695,2,1,1,1,2,1.0,1,1,1,2
696,5,10,10,3,7,3.0,8,10,2,4
697,4,8,6,4,3,4.0,10,6,1,4


In [37]:
#valores nulos hay en 'Bare_nuclei':
print("Valores nulos antes de imputar:", df_01['Bare_nuclei'].isna().sum())

Valores nulos antes de imputar: 16


In [38]:
#Imputar los valores nulos con la mediana :
mediana = df_01['Bare_nuclei'].median()
df_01['Bare_nuclei'] = df_01['Bare_nuclei'].fillna(mediana)

In [39]:
print("Valores nulos después de imputar:", df_01['Bare_nuclei'].isna().sum())


Valores nulos después de imputar: 0


In [40]:
print("Valores únicos antes del cambio:", df_01['Class'].unique())

Valores únicos antes del cambio: [2 4]


In [44]:
#Reemplazar los valores:
df_01['Class'] = df_01['Class'].replace({2: 0, 4: 1})

In [43]:
print("Valores únicos después del cambio:", df_01['Class'].unique())

Valores únicos después del cambio: [0 1]


In [45]:
# Creamos una copia para trabajar
df_clean_uni = df_01.copy()

# Seleccionamos solo las columnas numéricas (excepto 'Class')
numeric_cols = df_clean_uni.select_dtypes(include=[np.number]).drop(columns='Class').columns

# Eliminamos outliers univariados por IQR extendido (±3*IQR)
for col in numeric_cols:
    Q1 = df_clean_uni[col].quantile(0.25)
    Q3 = df_clean_uni[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 3 * IQR
    upper_bound = Q3 + 3 * IQR
    df_clean_uni = df_clean_uni[(df_clean_uni[col] >= lower_bound) & (df_clean_uni[col] <= upper_bound)]


In [46]:
from scipy.spatial.distance import mahalanobis

def remove_multivariate_outliers(df, threshold=30):
    df_copy = df.copy()

    # Seleccionar solo columnas numéricas (sin 'Class')
    features = df_copy.select_dtypes(include=[np.number]).drop(columns='Class', errors='ignore')

    # Asegurar que no haya columnas constantes
    features = features.loc[:, features.std() > 0]

    # Eliminar multicolinealidad (opcional, aquí no lo hacemos)

    x = features.values
    mean_vec = np.mean(x, axis=0)
    cov = np.cov(x, rowvar=False)
    inv_cov = np.linalg.pinv(cov)  # Pseudo-inversa por si es singular

    # Calcular distancia de Mahalanobis para cada fila
    distances = np.array([mahalanobis(row, mean_vec, inv_cov) for row in x])

    # Filtrar las filas dentro del umbral
    df_filtered = df_copy[distances <= threshold]
    return df_filtered

# Aplicamos al DataFrame limpio del paso anterior
df_final = remove_multivariate_outliers(df_clean_uni)


In [48]:
# Excluir la variable objetivo 'Class'
predictors = df_final.drop(columns='Class')

# Estadísticos descriptivos
estadisticos = predictors.describe()
estadisticos


,Clump_thickness,Uniformity_of_cell_size,Uniformity_of_cell_shape,Marginal_adhesion,Single_epithelial_cell_size,Bare_nuclei,Bland_chromatin,Normal_nucleoli,Mitoses
count,579.000000,579.000000,579.000000,579.000000,579.00000,579.000000,579.000000,579.000000,579.0
mean,3.851468,2.449050,2.578584,2.219344,2.75475,2.718480,3.003454,2.184801,1.0
std,2.528463,2.587315,2.550926,2.315565,1.80789,3.153268,2.199400,2.491239,0.0
min,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000,1.0
25%,2.000000,1.000000,1.000000,1.000000,2.00000,1.000000,1.500000,1.000000,1.0
50%,3.000000,1.000000,1.000000,1.000000,2.00000,1.000000,2.000000,1.000000,1.0
75%,5.000000,3.000000,3.000000,3.000000,3.00000,3.000000,3.000000,2.000000,1.0
max,10.000000,10.000000,10.000000,10.000000,10.00000,10.000000,10.000000,10.000000,1.0


In [49]:
from sklearn.preprocessing import StandardScaler

scaler_z = StandardScaler()
predictors_zscore = pd.DataFrame(
    scaler_z.fit_transform(predictors),
    columns=predictors.columns
)

# Añadir columna 'Class' de nuevo
df_zscore = pd.concat([predictors_zscore, df_final['Class'].reset_index(drop=True)], axis=1)


In [50]:
from sklearn.preprocessing import MinMaxScaler

scaler_minmax = MinMaxScaler()
predictors_minmax = pd.DataFrame(
    scaler_minmax.fit_transform(predictors),
    columns=predictors.columns
)

# Añadir columna 'Class' de nuevo
df_minmax = pd.concat([predictors_minmax, df_final['Class'].reset_index(drop=True)], axis=1)


In [ ]:
# Excluir la variable objetivo 'Class'
predictors = df_final.drop(columns='Class')

# Estadísticos descriptivos
estadisticos = predictors.describe()
estadisticos


In [52]:
# Excluir la columna 'Class' en ambos
desc_zscore = df_zscore.drop(columns='Class').describe().T
desc_minmax = df_minmax.drop(columns='Class').describe().T

# Cambiar nombres para identificar de dónde viene cada uno
desc_zscore.columns = [f'{col}_z' for col in desc_zscore.columns]
desc_minmax.columns = [f'{col}_minmax' for col in desc_minmax.columns]

# Unir las dos tablas en una sola para comparar
estadisticos_comparativos = pd.concat([desc_zscore, desc_minmax], axis=1)

# Mostrar
estadisticos_comparativos


,count_z,mean_z,std_z,min_z,25%_z,50%_z,75%_z,max_z,count_minmax,mean_minmax,std_minmax,min_minmax,25%_minmax,50%_minmax,75%_minmax,max_minmax
Clump_thickness,579.0,-1.012431e-16,1.000865,-1.128723,-0.732884,-0.337044,0.454634,2.433830,579.0,0.316830,0.280940,0.0,0.111111,0.222222,0.444444,1.0
Uniformity_of_cell_size,579.0,-2.454379e-17,1.000865,-0.560544,-0.560544,-0.560544,0.213127,2.920974,579.0,0.161006,0.287479,0.0,0.000000,0.000000,0.222222,1.0
Uniformity_of_cell_shape,579.0,-2.454379e-17,1.000865,-0.619363,-0.619363,-0.619363,0.165344,2.911819,579.0,0.175398,0.283436,0.0,0.000000,0.000000,0.222222,1.0
Marginal_adhesion,579.0,-1.288549e-16,1.000865,-0.527041,-0.527041,-0.527041,0.337426,3.363061,579.0,0.135483,0.257285,0.0,0.000000,0.000000,0.222222,1.0
Single_epithelial_cell_size,579.0,-7.363137e-17,1.000865,-0.971445,-0.417836,-0.417836,0.135773,4.011037,579.0,0.194972,0.200877,0.0,0.111111,0.111111,0.222222,1.0
Bare_nuclei,579.0,1.227190e-17,1.000865,-0.545455,-0.545455,-0.545455,0.089356,2.311194,579.0,0.190942,0.350363,0.0,0.000000,0.000000,0.222222,1.0
Bland_chromatin,579.0,-1.104471e-16,1.000865,-0.911697,-0.684166,-0.456635,-0.001572,3.183867,579.0,0.222606,0.244378,0.0,0.055556,0.111111,0.222222,1.0
Normal_nucleoli,579.0,9.817516e-17,1.000865,-0.475998,-0.475998,-0.475998,-0.074245,3.139785,579.0,0.131645,0.276804,0.0,0.000000,0.000000,0.111111,1.0
Mitoses,579.0,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,579.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0
